# Answering Business Questions Using SQL

In this scenario, we will act as a data analyst for the Chinook Music Store. We will be tasked with answering business questions, some based on hypothetical scenarios, for the store in order to help them increase their overall performance. The questions are the following:

* Which albums should the Chinook Store purchase based on the most popular genres?
* Which Sales Support Rep has the best sales performance?
* Which country has the highest sales amount for the Chinook Store?
* Should Chinook shift the way it allows customers to purchase music from their store?
* Which music artist is used in the most playlists?

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

***

# Overview of the Database

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Database Schema

![img](...%0AAABJRU5ErkJggg==)

## Overview of a few tables


In [3]:
%%sql
SELECT *
FROM employee
LIMIT 5;

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [4]:
%%sql
SELECT *
FROM customer
LIMIT 5;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [5]:
%%sql
SELECT *
FROM invoice
LIMIT 5;

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


***

# Scenario 1: Which albums should the Chinook Store purchase based on the most popular genres?

The Chinook record store has just signed a deal with a new record label, and the task is to select the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:


| Artist Name |	Genre
| :-----: | :-----: |
| Regal |	Hip-Hop
| Red Tone |	Punk
| Meteor and the Girls |	Pop
| Slim Jim Bites |	Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.


## New Albums to Purchase Based on Genre Popularity

In [6]:
%%sql
WITH genre_table as
    ( 

    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"


    )


SELECT
    g.name as Genre,
    count(gt.invoice_line_id) tracks_sold,
    cast(count(gt.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) from genre_table
    ) percentage_sold
FROM genre_table gt
INNER JOIN track t on t.track_id = gt.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;
    

Done.


Genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Based off how many tracks have been sold by each Genre in the U.S., the Chinook record store should select the following albums from the following artists:

* Red Tone
* Slim Jim Bites
* Meteor and the Girls

Earlier, a table was given with artist names and the corresponding genres for each artist that Chinook record store was interested in adding to their album selection. The table above with the number of U.S. tracks sold for certain genres was then used to line up with the potentially new artists and their corresponding genres. When matched up, the order of genre popularity based off tracks sold was the following:

1. Alternative & Punk
2. Blues
3. Pop

In light of this, the artist Regal did not make the cut since 3 artists needed to be selected and Hip Hop/Rap fell below Pop in order of popularity.


***

# Scenario 2: Which Sales Support Rep has the best sales performance?

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. The task is to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [7]:
%%sql

WITH employee_sales as 
(
    SELECT e.first_name || " " || e.last_name as Name, e.title as Title, e.city || ',' || ' ' || e.state as "City, State",
    e.country as Country, e.birthdate as Birthdate, i.total as Total FROM employee e
    INNER JOIN customer c ON c.support_rep_id = e.employee_id
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    
)

SELECT Name, Title, "City, State", Country, substr(Birthdate, 1, 4) as Birthdate, "$" || ROUND(SUM(Total), 2) as "Total Sales in Dollars"
FROM employee_sales
GROUP BY 1
ORDER BY 5 DESC;


Done.


Name,Title,"City, State",Country,Birthdate,Total Sales in Dollars
Jane Peacock,Sales Support Agent,"Calgary, AB",Canada,1973,$1731.51
Steve Johnson,Sales Support Agent,"Calgary, AB",Canada,1965,$1393.92
Margaret Park,Sales Support Agent,"Calgary, AB",Canada,1947,$1584.0


It is interesting to note that highest performing sales rep is also the youngest sales rep. A correlation might could be drawn between these two facts. One could argue that due to Jane Peacock being very young, she is in more intune with pop culture and the music industry. She knows what type of music people are listening to these days, therefore she knows what genre's will sell the best.

***

# Scenario 3: Where can the Chinook Store seek to expand their business?

## Analyze Sales by Country


This next task is to analyze the sales data for customers from each different country. The country value from the customers table will be used, and the country from the billing address in the invoice table ignored.

In particular, the following data will be calculated for each country:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

Because there are a number of countries with only one customer, these customers are grouped as "Other" in the analysis.

In [8]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );


Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


The lowest performing countries by sales are the Czech Republic, Portugal, India, and the United Kingdom. However, combining this with the least amount of customers per country and the Czech Republic, Portugal, and India rank at the bottom. These three countries should be where the Chinook Store puts their focus.

***

# Scenario 4: Should Chinook shift the way it allows customers to purchase music from their store?

## Background

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

* purchase a whole album
* purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

The task is to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [9]:
 %%sql

    WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Due to the low percentage of album purchases, Chinook should shift their strategy to focus on single tracks.

***

# Scenario 5: Which music artist is used in the most playlists?

In [10]:
%%sql

WITH artist_playlist AS
(
    SELECT a.name as artist_name, p.name as playlist FROM artist a
    INNER JOIN album al ON al.artist_id = a.artist_id
    INNER JOIN track t ON t.album_id = al.album_id
    INNER JOIN playlist_track pt ON pt.track_id = t.track_id
    INNER JOIN playlist p ON p.playlist_id = pt.playlist_id

)


SELECT artist_name, COUNT(DISTINCT(playlist)) as Count
FROM artist_playlist
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;


Done.


artist_name,Count
Eugene Ormandy,6
Academy of St. Martin in the Fields & Sir Neville Marriner,5
Berliner Philharmoniker & Herbert Von Karajan,5
English Concert & Trevor Pinnock,5
The King's Singers,5
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",4
Adrian Leaper & Doreen de Feis,4
Alberto Turco & Nova Schola Gregoriana,4
Antal Doráti & London Symphony Orchestra,4
Barry Wordsworth & BBC Concert Orchestra,4


It appears that the artist Eugene Ormandy is used in the most playlists.

***

# Conclusion

In this analysis, several business questions were answered for the Chinook Music Store:

* Which albums should the Chinook Store purchase based on the most popular genres? <br>
    __Answer__: Alternative & Punk, Blues, and Pop
    
    
* Which Sales Support Rep has the best sales performance? <br>
   __Answer__: Jane Peacock
   
    
* Which country has the highest sales amount for the Chinook Store? <br>
   __Answer__: Czech Republic, Portugal, and India
   

* Should Chinook shift the way it allows customers to purchase music from their store? <br>
   __Answer__: Due to the low percentage of album purchases, Chinook should shift their strategy to focus on single tracks.
   

* Which music artist is used in the most playlists? <br>
    __Answer__: Eugene Ormandy

